In [52]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import json
import re
from pydantic import BaseModel
from openai import OpenAI
import os

load_dotenv()
fireworks_key = os.getenv('FIREWORKS_API_KEY')

splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
train_df = pd.read_parquet(splits['train'])
test_df = pd.read_parquet(splits['test'])

In [53]:
q1_text = test_df['question'].iloc[0]
q1_text

"Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

In [ ]:
# Initialize the Fireworks client
client = OpenAI(
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY"),
)

# Define the output schema using Pydantic
class QAResult(BaseModel):
    question: str
    answer: str

# Prepare the user input
user_input = q1_text

# Construct the messages payload
messages = [{"role": "user", "content": user_input}]

# Make the API call to the model
response = client.chat.completions.create(
    model="accounts/fireworks/models/deepseek-r1",
    messages=messages,
    response_format={"type": "json_object", "schema": QAResult.model_json_schema()},
    max_tokens=1000,
)

# Extract the content of the response
response_content = response.choices[0].message.content
print("show response content", response_content)

# Extract the reasoning part enclosed in <think>...</think> tags
reasoning_match = re.search(r"<think>(.*?)</think>", response_content, re.DOTALL)
reasoning = reasoning_match.group(1).strip() if reasoning_match else "No reasoning provided."

# Extract the JSON part that follows after the reasoning
json_match = re.search(r"</think>\s*(\{.*\})", response_content, re.DOTALL)
json_str = json_match.group(1).strip() if json_match else "{}"

# Parse the JSON string directly into a Pydantic model
qa_result = QAResult.parse_raw(json_str)

# Output the reasoning and the parsed JSON data
print("Reasoning:")
print(reasoning)
print("\nQAResult (JSON Data):")
print(qa_result.json(indent=4))

show response content <think>
Okay, so I need to find out who wrote "Pride and Prejudice." Let me start by thinking about what I know. 

"Pride and Prejudice" is a classic novel, right? I remember hearing about it in school. It's one of those books that's often required reading. The story is set in England and revolves around themes of marriage, social status, and, obviously, pride and prejudice. The main characters are Elizabeth Bennet and Mr. Darcy.

Wait, the author... I think it's a female writer from the 19th century. Jane Austen? Yes, that sounds familiar. She wrote several other novels too, like "Sense and Sensibility," "Emma," and "Mansfield Park." But sometimes I get confused between the Brontë sisters and Jane Austen. The Brontës wrote "Jane Eyre" and "Wuthering Heights," which are also classics but more gothic and romantic compared to Austen's works, which are more about social commentary and satire.

Let me double-check the publication date. If "Pride and Prejudice" was pub

/var/folders/79/1660wkmj0r38z1dcqgl246980000gn/T/ipykernel_58007/3061839431.py:39: PydanticDeprecatedSince20: The `parse_raw` method is deprecated; if your data is JSON use `model_validate_json`, otherwise load the data then use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  qa_result = QAResult.parse_raw(json_str)
/var/folders/79/1660wkmj0r38z1dcqgl246980000gn/T/ipykernel_58007/3061839431.py:45: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(qa_result.json(indent=4))


TypeError: `dumps_kwargs` keyword arguments are no longer supported.

In [49]:
response.json()

{'id': 'e6f8be5d-fd59-4919-81ee-2206a4313c05',
 'object': 'text_completion',
 'created': 1740354387,
 'model': 'accounts/fireworks/models/deepseek-r1',
 'choices': [{'index': 0,
   'text': '\n\nOkay, let\'s see. So the problem is about Janet and her ducks. She gets eggs every day, uses some for breakfast and baking, and sells the rest. I need to figure out how much money she makes daily from selling the eggs. Let me break it down step by step.\n\nFirst, the problem says her ducks lay 16 eggs per day. Wait, but wait, the instruction says to replace every number with 15. So instead of 16 eggs, it\'s 15. Got it. So Janet\'s ducks lay 15 eggs each day. Then she eats three for breakfast every morning. But wait, the original number was three, but the instruction says replace every number with 15. Wait, hold on. Wait, the original problem had numbers 16, three, four, and two. The instruction says to replace every number with 15. So all numbers in the problem become 15. Let me check that again